Last update 31.07.2020

Author : Jeanne Colbois

Please send any comments, questions or remarks to Jeanne Colbois: jeanne.colbois@epfl.ch.
The author would appreciate to be cited in uses of this code, and would be very happy to hear about developments.

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
# Import modules
import numpy as np # maths

from time import time# timer for debug
    # gather results obtained from the runs  

import KagomeFunctions_OBC as kf # "library"< allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import KagomeFT as kft
import AnalysisFunctions_OBC as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt
import StartStates as strst
#import matplotlib.image as mpimg
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14})

In [ ]:
secondfoldername = ['../../Analysis_PSI/ExperimentalDataAndAnalysis_August2020/Inputs/']*11
secondfilename = ['PEEM_spin_config_threshold_0.txt',
                  'PEEM_spin_config_threshold_0.1.txt',
                  'PEEM_spin_config_threshold_0.2.txt',
                  'PEEM_spin_config_threshold_0.3.txt',
                  'PEEM_spin_config_threshold_0.4.txt',
                  'PEEM_spin_config_threshold_0.5.txt',
                  'PEEM_spin_config_threshold_0.6.txt',
                  'PEEM_spin_config_threshold_0.7.txt',
                  'PEEM_spin_config_threshold_0.8.txt',
                  'PEEM_spin_config_threshold_0.9.txt',
                  'PEEM_spin_config_threshold_1.txt']

In [ ]:
## Kagome lattice structure reader check
n = len(secondfilename)
nneighs2plot = 90
factors = [12]
nnloclist = [(11,12)]
distconds = [(0.95, 1.05), (1.68, 1.78), (1.95, 2.05), (2.59, 2.69), (2.95, 3.05), (3.41, 3.52), (3.54, 3.67), (3.95, 4.05)]
imgfile = ""

for i in range(1):
    print("------------------- File number ", i, " ----------------------")
    factor = factors[i]
    configlist, x, y, sconf= af.LoadSpinConfigsLarge(secondfoldername[i], 
                                                     secondfilename[i], 
                                                     imgfile, 
                                                     alpha = 1, 
                                                     factor = factor)
    x = 2*x;
    
    plt.plot(x[nnloclist[i][0]]/factor, y[nnloclist[i][0]]/factor,'.', color = "red")
    plt.plot(x[nnloclist[i][1]]/factor, y[nnloclist[i][1]]/factor,'.', color = "blue")
    print(x[nnloclist[i][0]]," ", y[nnloclist[i][0]], " ", x[nnloclist[i][1]], " ", y[nnloclist[i][1]])
    print(np.sqrt((x[nnloclist[i][0]]-x[nnloclist[i][1]])**2 + (y[nnloclist[i][0]]-y[nnloclist[i][1]])**2))
    distances_s1s2 = af.KagomeLatticeHistogram(x,y, factor=factor)
    plt.savefig(secondfoldername[i] + 'Histogram' + secondfilename[i]+'.png')
    plt.show()
    
    NNList = af.KagomeLatticeNeighboursLists(distances_s1s2, distconds)
    for j in range(4):
        fig, ax = plt.subplots(figsize = (8,8),dpi=200)
        af.plotSpinSites(secondfoldername, imgfile, x, y, 
                     [i for i in range(len(x))],
                     putimage = False, color = 'black', alpha = 0.3,
                     linestyle = 'none')
        for k in range(nneighs2plot):
            #print(list(NNList[j][k]))
            idk = np.random.randint(len(NNList[j]))
            af.plotSpinPairs(secondfoldername, imgfile, x, y,
                             [NNList[j][idk]], putimage = False, color = 'blue', alpha=0.3)
            
        plt.savefig(secondfoldername[i] + 'StructureSanityCheck_N{0}_'.format(j) + secondfilename[i]+'.png')
        plt.show()
    
        
    del configlist
    del x
    del y
    del sconf
    del distances_s1s2
    del NNList

In [ ]:
n = len(secondfilename)

#factors = [177, 178]
#nnloclist = [(11,12), (11,12)]

#factors = [178.34, 178.29, 178]
#nnloclist = [(11,12), (10,11), (11,12)]

#factors = [179, 179]
#nnloclist = [(10,11), (10,11)]
#
#factors = [179, 179, 180]
#nnloclist = [(10,11), (10,11), (11,12)]
#

#factors = [180, 177, 179]
#nnloclist = [(11,12), (11,12), (11,12)]

factors = [178.27, 178, 177, 178,
           177, 178, 178, 178,
           178, 176.5, 178, 176.5,
           179, 178.5, 178.5, 177,
           177, 178, 178.5, 178.5]
nnloclist = [(11,12), (11,12), (11,12), (11,12),
             (11,12), (11,12), (41,42), (41,42),
             (41,42), (41,42), (42,43), (42,43),
             (42,43), (42,43), (41,42), (41,42),
             (42,43), (41,42), (42,43), (42,43)]
#factors=[178, 176.5,
#           179, 178.5, 178.5, 177,
#           177, 178, 178.5, 178.5]
#nnloclist = [(42,43), (42,43),
#             (42,43), (42,43), (41,42), (41,42),
#             (42,43), (41,42), (42,43), (42,43)]
#
distconds = [(0.95, 1.05), (1.68, 1.78), (1.95, 2.05), (2.59, 2.69), (2.95, 3.05), (3.41, 3.52), (3.54, 3.67), (3.95, 4.05)]
imgfile = ""

covlist = []
varcovlist = []
dlist = []
vardlist = []
mlist = []

print(len(factors))

for i in range(len(secondfilename)):
    print("------------------- File number ", i, " ----------------------")
    factor = factors[i]
    configlist, x, y, sconf= af.LoadSpinConfigsLarge(secondfoldername[i], 
                                                     secondfilename[i], 
                                                     imgfile, 
                                                     alpha = 1, 
                                                     factor = factor)

    plt.plot(x[nnloclist[i][0]]/factor, y[nnloclist[i][0]]/factor,'.', color = "red")
    plt.plot(x[nnloclist[i][1]]/factor, y[nnloclist[i][1]]/factor,'.', color = "blue")
    print(x[nnloclist[i][0]]," ", y[nnloclist[i][0]], " ", x[nnloclist[i][1]], " ", y[nnloclist[i][1]])
    print(np.sqrt((x[nnloclist[i][0]]-x[nnloclist[i][1]])**2 + (y[nnloclist[i][0]]-y[nnloclist[i][1]])**2))
    m = sum(sconf)/len(sconf)
    print("magnetisation: ", m)
    mlist.append(m)
    distances_s1s2 = af.KagomeLatticeHistogram(x,y, factor=factor)
    plt.savefig(secondfoldername[i] + 'Histogram' + secondfilename[i]+'.png')
    plt.show()
    
    NNList = af.KagomeLatticeNeighboursLists(distances_s1s2, distconds)
    cconf, xc,yc, tlist = af.KagomeLatticeCharges(NNList,sconf,x,y)
    charges = [np.count_nonzero(cconf == 3), np.count_nonzero(cconf == 1),
           np.count_nonzero(cconf == -1), np.count_nonzero(cconf == -3)]
    print(charges)
    np.savetxt(secondfoldername[i] + 'charges' + secondfilename[i],
           np.array(charges), newline=",")
    
    fig, ax = plt.subplots(figsize = (8,8),dpi=200)
    af.plotSpinSites(secondfoldername, imgfile, x, y, 
                 [i for i in range(len(x)) if sconf[i] == 1],
                 putimage = False, color = 'blue', alpha = 1,
                 linestyle = 'none')
    af.plotSpinSites(secondfoldername, imgfile, x, y, 
                     [i for i in range(len(x)) if sconf[i] == -1],
                     putimage = False, color = 'red', alpha = 1,
                     linestyle = 'none')
    af.plotSpinSites(secondfoldername, imgfile, xc, yc, 
                     [i for i in range(len(xc)) if abs(cconf[i]) == 3], 
                     putimage = False, color = 'yellow', alpha = 1,
                     linestyle = 'none')
    #af.plotSpinSites(secondfoldername, imgfile, xc, yc, 
    #                 [i for i in range(len(xc)) if abs(cconf[i]) == 1], 
    #                 putimage = False, color = 'green', alpha = 1,
    #                 linestyle = 'none')
    plt.savefig(secondfoldername[i] + 'State_' + secondfilename[i]+'.png')
    plt.show()
    
    cov = []
    varcov = []
    d = []
    vard = []

    for j in range(9):
        print(j)
        cov0, varcov0, d0, vard0= af.ComputeNthCorrList(NNList[j], sconf, x, y, factor)
        cov.append(cov0)
        varcov.append(varcov0)
        d.append(d0)
        vard.append(vard0)
      
    covlist.append(cov)
    varcovlist.append(varcov)
    dlist.append(d)
    vardlist.append(vard)
    
    
        
    del configlist
    del x
    del y
    del sconf
    del distances_s1s2
    del NNList
    del cconf
    del xc
    del yc
    del charges
    del cov
    del varcov
    del d
    del vard


In [ ]:
mlist = np.array(mlist)
avgm = np.sum(mlist)/20
print(np.std(mlist))

In [ ]:
avgm

In [ ]:
d = np.array(dlist).sum(0)/n
vard = np.array(vardlist).sum(0)/n

cov = np.array(covlist).sum(0)/n
varcov = np.array([(covlist[i] - cov)**2 for i in range(n)]).sum(0)/n

varcov2 = np.array(varcovlist).sum(0)/n

#### Bootstrapping the result to get an error estimate:
The idea is to compute the correlations as the average of the correlations over 18 of the 20 samples, chosing these 18 in an arbitrary way, then compute the error as the standard deviation of this result

In [ ]:
np.array(covlist).shape

In [ ]:
avgcorr = []
corr = np.array(covlist)
s = 5
nsample = 20
masks = []
for randid in range(nsample):
    mask = np.random.choice(20, size=s)
    masks.append(mask)
    avgcorr.append(corr[mask,:,:].sum(0)/s)
    
avgcorr = np.array(avgcorr)
navg = avgcorr.shape[0]
stdev = np.std(avgcorr, axis=0)
avgcorrplot = avgcorr.sum(0)/nsample

In [ ]:
stdev

In [ ]:
avgcorrplot

In [ ]:
ListNei = ['1', '2', '3par', '3star', '4', '5', '6', '6bis', '7', '8', '8bis','9']
fig, ax = plt.subplots(figsize = (8,8), dpi=200)
for index in range(4):
    print(index)
    plt.errorbar(1, avgcorrplot[index,0],yerr=stdev[index,0], 
    label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
plt.xlabel('')
plt.ylabel('Accumulated correlations')
plt.title('20 samples average,\n errors from std deviation of {0} samples averages'.format(s))
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+secondfilename[0]+'.png')    

In [ ]:
fig, ax = plt.subplots(figsize = (8,8), dpi=200)
for i in range(5):
    plt.gca().set_prop_cycle(None)
    for index in range(4):
        print(index)
        if i == 0:
            plt.errorbar(i, avgcorr[i,index,0],yerr=stdev[index,0], 
            label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
        else:
            plt.errorbar(i, avgcorr[i,index,0],yerr=stdev[index,0],  fmt= 'x')
plt.xticks([0, 1, 2, 3, 4], ['Set 1', 'Set 2', 'Set 3','Set 4','Set 5'])
plt.ylabel('Accumulated correlations')
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+'_5Masks_{1}_samples_bootstrapping'.format(i,s)+secondfilename[0]+'.png')    

In [ ]:
print(cov)

In [ ]:
print(np.sqrt(np.array(varcov)))
print(np.sqrt(np.array(varcov2)))

In [ ]:
fig, ax = plt.subplots(dpi=200)
plt.errorbar(d, np.array(cov),xerr = np.sqrt(np.array(vard)),yerr=np.sqrt(np.array(varcov)),  fmt= '.')
plt.xlabel('Distance')
plt.ylabel('Accumulated correlations')
plt.ylim(-0.27,0.12)
plt.savefig(secondfoldername[i] + 'FirstResult_' + secondfilename[i]+'.png')
plt.show()

In [ ]:
ListNei = ['1', '2', '3par', '3star', '4', '5', '6', '6bis', '7', '8', '8bis','9']
for i in range(n):
    fig, ax = plt.subplots(figsize = (8,8), dpi=200)

    for index, covi in enumerate(covlist[i][0:5]):
        #print(index)
        plt.errorbar(1, covi,yerr=np.sqrt(np.array(varcovlist[i])[index]), label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')

    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('Accumulated correlations')
    plt.ylim(-0.27, 0.12)
    plt.legend(loc='best')
    plt.savefig(secondfoldername[i]+'FirstFewCorrelations'+secondfilename[i]+'.png')

fig, ax = plt.subplots(figsize = (8,8),dpi=200)
x = [1 for y in cov];

for index, covi in enumerate(cov[0:4]):
    print(index)
    plt.errorbar(1, covi,yerr=np.sqrt(np.array(varcov2)[index]), label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
plt.xlabel('')
plt.xticks([])
plt.ylabel('Accumulated correlations (20 samples average)')
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+secondfilename[0]+'.png')    

In [ ]:
#ssecondfoldername = '../Analysis_PSI/Kagome_54-1_Demag_all/'
#ssecondfilename = '54-1_Demag7_5-TR_spin_config.txt'
#iimgfile = ""
#ffactor = 178.7
#cconfiglist, x, y, sconf = af.LoadSpinConfigsLarge(secondfoldername, secondfilename, imgfile, alpha = 1, factor = factor)
#pplt.plot(x[43]/factor, y[43]/factor,'.', color = "red")
#pplt.plot(x[42]/factor, y[42]/factor,'.', color = "blue")
#pprint(x[43]," ", y[10], " ", x[11], " ", y[11])
#pprint(np.sqrt((x[43]-x[42])**2 + (y[43]-y[42])**2))
#mm = sum(sconf)/len(sconf)
#pprint(m)